In [1]:
#!/usr/bin/env python3

import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # Not supported in PyTorch < 2.0
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    BertForMaskedLM, GPT2LMHeadModel, GPT2TokenizerFast
)
from sentence_transformers import SentenceTransformer, util
from lime.lime_text import LimeTextExplainer
import shap
from difflib import SequenceMatcher
from itertools import combinations # Added for multi-token combinations

/home2/hardik.mittal/miniconda3/envs/new_storysumm/lib/python3.8/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
# ----------------------------------------------------------------------
# Configuration
# ----------------------------------------------------------------------

# !! IMPORTANT !! Change this to your own cache directory !!
cache_dir = "/ssd_scratch/sweta.jena"  # <-- CHANGE THIS
dataset_name="imdb"
class_names = ["negative", "positive"]
# sim_threshold=0.85
# ppl_threshold=100

In [3]:
# ----------------------------------------------------------------------
# Model Loading
# ----------------------------------------------------------------------

print("Loading models...")

# sentiment clf
clf_name = "textattack/bert-base-uncased-imdb"    
clf_tokenizer = AutoTokenizer.from_pretrained(clf_name, cache_dir = cache_dir)
clf_model = AutoModelForSequenceClassification.from_pretrained(clf_name, cache_dir=cache_dir)
clf_model.eval()

# masked LM
mlm_name = "bert-base-uncased"                  
mlm_tokenizer = AutoTokenizer.from_pretrained(mlm_name, cache_dir = cache_dir)
mlm_model = BertForMaskedLM.from_pretrained(mlm_name, cache_dir=cache_dir)
mlm_model.eval()


# fluency
gpt2_name = "gpt2"                            
gpt2_tokenizer = GPT2TokenizerFast.from_pretrained(gpt2_name, cache_dir = cache_dir)
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token
gpt2_model = GPT2LMHeadModel.from_pretrained(gpt2_name, cache_dir=cache_dir)
gpt2_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    clf_model = torch.nn.DataParallel(clf_model)
    mlm_model = torch.nn.DataParallel(mlm_model)
    gpt2_model = torch.nn.DataParallel(gpt2_model)
clf_model.to(device)
mlm_model.to(device)
gpt2_model.to(device)

#semantic sim
sbert = SentenceTransformer("all-MiniLM-L6-v2", device=str(device))

print("Models loaded successfully.")

Loading models...


BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

Models loaded successfully.


In [4]:
# ----------------------------------------------------------------------
# Helper Functions
# ----------------------------------------------------------------------

def predict_proba(texts, batch_size=64):
    
    processed_texts = []
    for t in texts:
        if isinstance(t, (list, np.ndarray)):
            processed_texts.append(clf_tokenizer.decode(t, skip_special_tokens=True))
        else:
            processed_texts.append(str(t))

    all_probs = []
    for i in range(0, len(processed_texts), batch_size):
        batch = processed_texts[i:i+batch_size]
        encodings = clf_tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = clf_model(**encodings)
        probs = torch.softmax(outputs.logits, dim=1).detach().cpu().numpy()
        all_probs.extend(probs)
    return np.array(all_probs)

def generate_candidates(text, word, top_k=10):
    tokens = mlm_tokenizer.tokenize(text, max_length=512, truncation=True)
    if word not in tokens:
        # Try to find the word even if it's been tokenized differently by SHAP
        # This is a fallback, but the first-subword-strategy in batch_evaluate is the main one
        found = False
        for i, tok in enumerate(tokens):
            if tok.startswith(word):
                word = tok
                found = True
                break
        if not found:
            # print(f"Warning: Token '{word}' not in tokenized text.")
            return []
    
    try:
        idx = tokens.index(word)
    except ValueError:
        # print(f"Warning: Token '{word}' not found despite check.")
        return []

    tokens[idx] = mlm_tokenizer.mask_token
    masked_text = mlm_tokenizer.convert_tokens_to_string(tokens)
    inputs = mlm_tokenizer(masked_text, return_tensors="pt", max_length=512, truncation=True).to(device)

    with torch.no_grad():
        logits = mlm_model(**inputs).logits
    mask_index = torch.where(inputs["input_ids"][0] == mlm_tokenizer.mask_token_id)[0].item()
    probs = torch.softmax(logits[0, mask_index], dim=0)
    top_tokens = torch.topk(probs, top_k).indices.tolist()
    candidates = [mlm_tokenizer.decode([t]) for t in top_tokens]
    return candidates

def semantic_similarity(text1, text2):
    emb1 = sbert.encode(text1, convert_to_tensor=True)
    emb2 = sbert.encode(text2, convert_to_tensor=True)
    return float(util.cos_sim(emb1, emb2))

def perplexity(text):

    model_config = gpt2_model.module.config if hasattr(gpt2_model, "module") else gpt2_model.config
    max_length = model_config.n_positions
    encodings = gpt2_tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length).to(device)
    stride = 512

    lls = []
    for i in range(0, encodings.input_ids.size(1), stride):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - i
        input_ids = encodings.input_ids[:, begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100
        with torch.no_grad():
            outputs = gpt2_model(input_ids, labels=target_ids)
            log_likelihood = outputs.loss * trg_len
        lls.append(log_likelihood)
    ppl = torch.exp(torch.stack(lls).sum() / end_loc)
    return float(ppl)

def edit_distance(a, b):
    return 1 - SequenceMatcher(None, a.split(), b.split()).ratio()

# Placeholder function for Attribute Isolation Logic
def check_attribute_isolation(text_a, text_b, isolation_classifier=None, isolation_tokenizer=None, isolation_label=None):
    """
    Placeholder for attribute isolation logic.
    In a real multi-attribute task (e.g., sentiment + topic), you would:
    1. Load a separate classifier trained on the non-target attribute (e.g., a topic classifier).
    2. Tokenize both text_a (original) and text_b (counterfactual).
    3. Get predictions from the isolation_classifier for both texts.
    4. Return True if the predictions are the same, False otherwise.
    
    Since the IMDb dataset is single-attribute (sentiment), we'll just return True.
    """
    # Example logic you would implement:
    # if isolation_classifier is not None:
    #     inputs_a = isolation_tokenizer(text_a, return_tensors="pt", truncation=True, padding=True).to(device)
    #     inputs_b = isolation_tokenizer(text_b, return_tensors="pt", truncation=True, padding=True).to(device)
    #     with torch.no_grad():
    #         pred_a = torch.argmax(isolation_classifier(**inputs_a).logits, dim=1).item()
    #         pred_b = torch.argmax(isolation_classifier(**inputs_b).logits, dim=1).item()
    #     return pred_a == pred_b
    
    # For this project using IMDb, we assume isolation is met.
    return True

def evaluate_counterfactual(cf, orig_text, method, sim_threshold=0.75, ppl_threshold=200):
    if cf is None:
        return {
            "success": -1,
            "method": method,
            "original_text": orig_text,
            "counterfactual_text": None,
            "changed_word": None,
            "semantic_similarity": None,
            "perplexity": None,
            "edit_distance": None,
            "original_embedding": None,
            "counterfactual_embedding": None,
            "mced": None,
            "edit_type": cf.get("edit_type", "single-token") # track edit type
        }

    orig_emb = sbert.encode(orig_text, convert_to_tensor=False)
    cf_emb = sbert.encode(cf.get("counterfactual_text"), convert_to_tensor=False)
    ed = edit_distance(orig_text, cf.get("counterfactual_text"))
    mced = ed / max(1, len(orig_text.split()))  # normalized edit distance

    if (cf.get("semantic_similarity") >= sim_threshold) and (cf.get("perplexity") <= ppl_threshold):
        success = 1
    else:
        success = 0

    return {
        "success": success,
        "method": method,
        "original_text": cf.get("original_text", orig_text),
        "counterfactual_text": cf.get("counterfactual_text"),
        "changed_word": cf.get("changed_word"),
        "semantic_similarity": cf.get("semantic_similarity"),
        "perplexity": cf.get("perplexity"),
        "edit_distance": ed,
        "original_embedding": orig_emb.tolist(),      
        "counterfactual_embedding": cf_emb.tolist(),
        "mced": mced,
        "edit_type": cf.get("edit_type", "single-token") # track edit type
    }

In [5]:
# ----------------------------------------------------------------------
# Main Evaluation Function
# ----------------------------------------------------------------------

def batch_evaluate(
    sample_size=-1,
    dataset_name="imdb",
    batch_size=64,
    max_len=512,
    lime_num_samples=500,
    shap_max_evals=500,
    multi_token_top_k=3 # Smaller top_k for multi-token combos to manage speed
):
    dataset = load_dataset(dataset_name)
    test_data = dataset["test"]

    if sample_size == -1:
        sample_size = len(test_data)
        examples=list(test_data)
        
    else:
        examples = random.sample(list(test_data), sample_size)

    print("Sample size:", sample_size)

    results = []

    masker = shap.maskers.Text(tokenizer=clf_tokenizer)
    model_config = clf_model.module.config if hasattr(clf_model, "module") else clf_model.config
    labels = [model_config.id2label[i] for i in range(len(model_config.id2label))]
    shap_explainer = shap.Explainer(predict_proba, masker, output_names=labels)

    lime_explainer = LimeTextExplainer(class_names=class_names)

    for i in range(0, len(examples), batch_size):
        batch = examples[i:i+batch_size]
        texts = []
        for ex in batch:
            enc = clf_tokenizer(
                ex["text"],
                truncation=True,
                padding=False,
                max_length=max_len,
                return_tensors="pt"
            )
            truncated_text = clf_tokenizer.decode(enc["input_ids"][0], skip_special_tokens=True)
            texts.append(truncated_text)

        
        # SHAP counterfactuals ####################################
        
        try:
            shap_values = shap_explainer(texts, max_evals=shap_max_evals)
        except Exception as e:
            print(f"SHAP explanation failed for batch {i}: {e}")
            continue

        for j, text in enumerate(texts):
            try:
                pred_label = np.argmax(predict_proba([text])[0])
                
                vals = shap_values.values[j]
                tokens = shap_values.data[j]

                # handle multi-class output
                if vals.ndim == 1:
                    token_importances = vals
                else:
                    token_importances = vals[:, pred_label]

                top_indices = np.argsort(np.abs(token_importances))[-5:]
                influential_tokens_shap = [tokens[idx].strip() for idx in top_indices if len(tokens[idx].strip()) > 0]

                # --- Start of Single-Token Edit Logic (Original) ---
                cands_by_word_shap = {}
                for target_word in influential_tokens_shap:
                    subword = mlm_tokenizer.tokenize(target_word)
                    if len(subword) == 0:
                        cands_by_word_shap[target_word] = []
                        continue
                    subword = subword[0]
                    candidates = generate_candidates(text, subword, top_k=10) # Original top_k
                    cands_by_word_shap[target_word] = candidates
                    
                    for cand in candidates:
                        new_text = text.replace(target_word, cand)
                        new_pred = np.argmax(predict_proba([new_text])[0])
                        
                        # Added Attribute Isolation Check
                        isolated = check_attribute_isolation(text, new_text)

                        if new_pred != pred_label and isolated:
                            sim = semantic_similarity(text, new_text)
                            flu = perplexity(new_text)
                            cf_shap = {
                                "original_text": text,
                                "original_pred": pred_label,
                                "counterfactual_text": new_text,
                                "counterfactual_pred": new_pred,
                                "changed_word": (target_word, cand),
                                "semantic_similarity": sim,
                                "perplexity": flu,
                                "edit_type": "single-token" # Added edit_type
                            }
                            metrics = evaluate_counterfactual(cf_shap, text, method="SHAP")
                            results.append(metrics)
                # --- End of Single-Token Edit Logic ---

                # --- NEW: Start of Multi-Token Edit (2-token) Logic ---
                for word1, word2 in combinations(cands_by_word_shap.keys(), 2):
                    # Use smaller k for combinations to avoid combinatorial explosion
                    for cand1 in cands_by_word_shap[word1][:multi_token_top_k]:
                        for cand2 in cands_by_word_shap[word2][:multi_token_top_k]:
                            # Apply replaces sequentially.  
                            new_text = text.replace(word1, cand1).replace(word2, cand2)
                            new_pred = np.argmax(predict_proba([new_text])[0])
                            isolated = check_attribute_isolation(text, new_text)

                            if new_pred != pred_label and isolated:
                                sim = semantic_similarity(text, new_text)
                                flu = perplexity(new_text)
                                cf_shap_multi = {
                                    "original_text": text,
                                    "original_pred": pred_label,
                                    "counterfactual_text": new_text,
                                    "counterfactual_pred": new_pred,
                                    "changed_word": (f"{word1}->{cand1}", f"{word2}->{cand2}"),
                                    "semantic_similarity": sim,
                                    "perplexity": flu,
                                    "edit_type": "multi-token (2)"
                                }
                                metrics = evaluate_counterfactual(cf_shap_multi, text, method="SHAP-Multi-2")
                                results.append(metrics)
                # --- NEW: End of Multi-Token Edit (2-token) Logic ---
            
            except Exception as e:
                print(f"Error processing text {j} in SHAP batch {i}: {e}")
                continue
        
        pd.DataFrame(results).to_csv(cache_dir+f"/intermediate_counterfactuals_{i}_shap.csv", index=False)

        # LIME counterfactuals ####################################
    
        for text in texts:
            try:
                exp = lime_explainer.explain_instance(
                    text,
                    predict_proba,
                    num_features=5,
                    labels=[0, 1],
                    num_samples=lime_num_samples
                )

                pred_label = np.argmax(predict_proba([text])[0])
                influential_tokens_lime = [w for w, score in sorted(exp.as_list(label=pred_label), key=lambda x: abs(x[1]), reverse=True)][:5]
                if not influential_tokens_lime:
                    continue

                # --- Start of Single-Token Edit Logic (Original) ---
                cands_by_word_lime = {}
                for target_word in influential_tokens_lime:
                    subword = mlm_tokenizer.tokenize(target_word)
                    if len(subword) == 0:
                        cands_by_word_lime[target_word] = []
                        continue
                    subword = subword[0]
                    candidates = generate_candidates(text, subword, top_k=10)
                    cands_by_word_lime[target_word] = candidates

                    for cand in candidates:
                        new_text = text.replace(target_word, cand)
                        new_pred = np.argmax(predict_proba([new_text])[0])
                        
                        # Added Attribute Isolation Check
                        isolated = check_attribute_isolation(text, new_text)
                        
                        if new_pred != pred_label and isolated:
                            sim = semantic_similarity(text, new_text)
                            flu = perplexity(new_text)
                            cf_lime = {
                                "original_text": text,
                                "original_pred": pred_label,
                                "counterfactual_text": new_text,
                                "counterfactual_pred": new_pred,
                                "changed_word": (target_word, cand),
                                "semantic_similarity": sim,
                                "perplexity": flu,
                                "edit_type": "single-token" # Added edit_type
                            }
                            metrics = evaluate_counterfactual(cf_lime, text, method="LIME")
                            results.append(metrics)
                # --- End of Single-Token Edit Logic ---

                # --- NEW: Start of Multi-Token Edit (2-token) Logic ---
                for word1, word2 in combinations(cands_by_word_lime.keys(), 2):
                    for cand1 in cands_by_word_lime[word1][:multi_token_top_k]:
                        for cand2 in cands_by_word_lime[word2][:multi_token_top_k]:
                            new_text = text.replace(word1, cand1).replace(word2, cand2)
                            new_pred = np.argmax(predict_proba([new_text])[0])
                            isolated = check_attribute_isolation(text, new_text)

                            if new_pred != pred_label and isolated:
                                sim = semantic_similarity(text, new_text)
                                flu = perplexity(new_text)
                                cf_lime_multi = {
                                    "original_text": text,
                                    "original_pred": pred_label,
                                    "counterfactual_text": new_text,
                                    "counterfactual_pred": new_pred,
                                    "changed_word": (f"{word1}->{cand1}", f"{word2}->{cand2}"),
                                    "semantic_similarity": sim,
                                    "perplexity": flu,
                                    "edit_type": "multi-token (2)"
                                }
                                metrics = evaluate_counterfactual(cf_lime_multi, text, method="LIME-Multi-2")
                                results.append(metrics)
                # --- NEW: End of Multi-Token Edit (2-token) Logic ---
            
            except Exception as e:
                print(f"Error processing text in LIME batch {i}: {e}")
                continue

        pd.DataFrame(results).to_csv(cache_dir+f"/intermediate_counterfactuals_{i}_shap_lime.csv", index=False)
        print(f"Processed {min(i + batch_size, sample_size)}/{sample_size} examples")

    df = pd.DataFrame(results)
    df.to_csv("all_counterfactuals.csv", index=False)

    summary = df[df["success"] == 1].groupby("method").mean(numeric_only=True).to_dict()

    return df, summary

In [ ]:
# ----------------------------------------------------------------------
# Script Execution
# ----------------------------------------------------------------------

if __name__ == "__main__":
    print("Starting counterfactual generation and evaluation...")
    
    # To run on a small sample for testing, change sample_size:
    # df, summary = batch_evaluate(sample_size=10, dataset_name=dataset_name)
    
    # To run on the full dataset (as in the notebook):
    df, summary = batch_evaluate(sample_size=-1, dataset_name=dataset_name)
    
    print("\n--- Evaluation Summary (Successful Counterfactuals) ---")
    print(summary)
    print("\nProcess complete. Results saved to 'all_counterfactuals.csv'.")

Starting counterfactual generation and evaluation...


Sample size: 25000


Partition explainer:   6%|▋         | 4/64 [00:14<01:04,  1.07s/it]

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer:  11%|█         | 7/64 [00:26<02:53,  3.04s/it]

...

Partition explainer:  91%|█████████ | 58/64 [04:08<00:28,  4.68s/it]

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer:  92%|█████████▏| 59/64 [04:14<00:25,  5.18s/it]

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer: 65it [04:38,  4.56s/it]                        



In [1]:
# Load and display the latest intermediate results
import pandas as pd

latest_file = "/ssd_scratch/sweta.jena/intermediate_counterfactuals_896_shap.csv"
df = pd.read_csv(latest_file)

print(f"File: {latest_file}")
print(f"Total rows: {len(df)}")
print(f"Columns: {list(df.columns)}")
print("\n" + "="*80)
print("First 10 rows:")
print("="*80)
df.head(10)

File: /ssd_scratch/sweta.jena/intermediate_counterfactuals_896_shap.csv
Total rows: 11903
Columns: ['success', 'method', 'original_text', 'counterfactual_text', 'changed_word', 'semantic_similarity', 'perplexity', 'edit_distance', 'original_embedding', 'counterfactual_embedding', 'mced', 'edit_type']

First 10 rows:


,success,method,original_text,counterfactual_text,changed_word,semantic_similarity,perplexity,edit_distance,original_embedding,counterfactual_embedding,mced,edit_type
0,1,SHAP,"worth the entertainment value of a rental, esp...","worth the entertainment value of a rental, esp...","('.', 'but')",0.970543,68.212982,0.054393,"[0.013422119431197643, -0.05341522768139839, 0...","[0.01939862035214901, -0.0657210573554039, -0....",0.000228,single-token
1,1,SHAP,"worth the entertainment value of a rental, esp...","worth the entertainment value of a rental, esp...","('.', 'and')",0.978007,73.863052,0.054393,"[0.013422119431197643, -0.05341522768139839, 0...","[0.015377247706055641, -0.07202999293804169, 0...",0.000228,single-token
2,1,SHAP-Multi-2,"worth the entertainment value of a rental, esp...","worth the entertainment value of a rental, esp...","('.->...', 'not->very')",0.938516,58.596718,0.104603,"[0.013422119431197643, -0.05341522768139839, 0...","[0.00359835266135633, -0.054686449468135834, 0...",0.000438,multi-token (2)
3,1,SHAP,its a totally average film with a few semi - a...,its a totally average film with a few semi - a...,"('not', 'is')",0.996635,58.650810,0.007407,"[-0.04979102686047554, 0.030777275562286377, -...","[-0.05127647519111633, 0.02394253946840763, -0...",0.000055,single-token
4,1,SHAP,its a totally average film with a few semi - a...,its a totally average film with a few semi - a...,"('not', 'was')",0.999207,59.027637,0.007407,"[-0.04979102686047554, 0.030777275562286377, -...","[-0.049222689121961594, 0.028409535065293312, ...",0.000055,single-token
5,1,SHAP,its a totally average film with a few semi - a...,its a totally average film with a few semi - a...,"('not', 'bit')",0.999036,62.394833,0.007407,"[-0.04979102686047554, 0.030777275562286377, -...","[-0.04825850576162338, 0.03049815073609352, -0...",0.000055,single-token
6,1,SHAP,its a totally average film with a few semi - a...,its a totally average film with a few semi - a...,"('not', 'its')",0.999431,59.807793,0.007407,"[-0.04979102686047554, 0.030777275562286377, -...","[-0.04866982251405716, 0.02756422385573387, -0...",0.000055,single-token
7,1,SHAP,its a totally average film with a few semi - a...,its a totally average film with a few semi - a...,"('not', 'it')",0.999312,60.539921,0.007407,"[-0.04979102686047554, 0.030777275562286377, -...","[-0.048071280121803284, 0.02763962559401989, -...",0.000055,single-token
8,1,SHAP,its a totally average film with a few semi - a...,its a totally average film with a few semi - a...,"('not', 'but')",0.999286,60.959511,0.007407,"[-0.04979102686047554, 0.030777275562286377, -...","[-0.050051577389240265, 0.029488245025277138, ...",0.000055,single-token
9,1,SHAP-Multi-2,its a totally average film with a few semi - a...,its a totally average film with a few semi - a...,"('little->lot', 'not->is')",0.995233,65.385201,0.022222,"[-0.04979102686047554, 0.030777275562286377, -...","[-0.05124744772911072, 0.021253619343042374, -...",0.000165,multi-token (2)


In [2]:
# Summary statistics
print("\n" + "="*80)
print("SUMMARY STATISTICS")
print("="*80)
print(f"\nTotal counterfactuals generated: {len(df)}")
print(f"Successful counterfactuals: {df['success'].sum()}")
print(f"Success rate: {df['success'].mean()*100:.2f}%")

print(f"\nBy Method:")
print(df.groupby('method')['success'].agg(['count', 'sum', 'mean']))

print(f"\nBy Edit Type:")
print(df.groupby('edit_type')['success'].agg(['count', 'sum', 'mean']))

print(f"\nSemantic Similarity Stats (for successful counterfactuals):")
successful = df[df['success'] == 1]
if len(successful) > 0:
    print(f"  Mean: {successful['semantic_similarity'].mean():.4f}")
    print(f"  Median: {successful['semantic_similarity'].median():.4f}")
    print(f"  Min: {successful['semantic_similarity'].min():.4f}")
    print(f"  Max: {successful['semantic_similarity'].max():.4f}")

print(f"\nPerplexity Stats (for successful counterfactuals):")
if len(successful) > 0:
    print(f"  Mean: {successful['perplexity'].mean():.2f}")
    print(f"  Median: {successful['perplexity'].median():.2f}")
    print(f"  Min: {successful['perplexity'].min():.2f}")
    print(f"  Max: {successful['perplexity'].max():.2f}")


SUMMARY STATISTICS

Total counterfactuals generated: 11903
Successful counterfactuals: 10506
Success rate: 88.26%

By Method:
              count   sum      mean
method                             
LIME           1551  1408  0.907801
LIME-Multi-2   4773  4361  0.913681
SHAP           1496  1280  0.855615
SHAP-Multi-2   4083  3457  0.846681

By Edit Type:
                 count   sum      mean
edit_type                             
multi-token (2)   8856  7818  0.882791
single-token      3047  2688  0.882179

Semantic Similarity Stats (for successful counterfactuals):
  Mean: 0.9765
  Median: 0.9926
  Min: 0.7501
  Max: 1.0000

Perplexity Stats (for successful counterfactuals):
  Mean: 62.96
  Median: 54.80
  Min: 13.28
  Max: 199.76


In [3]:
# Show some example counterfactuals
print("\n" + "="*80)
print("SAMPLE SUCCESSFUL COUNTERFACTUALS")
print("="*80)

successful_samples = df[df['success'] == 1].sample(min(5, len(df[df['success'] == 1])))

for idx, row in successful_samples.iterrows():
    print(f"\n{'─'*80}")
    print(f"Method: {row['method']} | Edit Type: {row['edit_type']}")
    print(f"Changed Word: {row['changed_word']}")
    print(f"Similarity: {row['semantic_similarity']:.4f} | Perplexity: {row['perplexity']:.2f}")
    print(f"\nOriginal: {row['original_text'][:200]}...")
    print(f"\nCounterfactual: {row['counterfactual_text'][:200]}...")
    print(f"{'─'*80}")


SAMPLE SUCCESSFUL COUNTERFACTUALS

────────────────────────────────────────────────────────────────────────────────
Method: LIME-Multi-2 | Edit Type: multi-token (2)
Changed Word: ('disappointing->scary', 'bad->great')
Similarity: 0.9632 | Perplexity: 92.50

Original: technically i'am a van damme fan, or i was. this movie is so bad that i hated myself for wasting those 90 minutes. do not let the name isaac florentine ( undisputed ii ) fool you, i had big hopes for ...

Counterfactual: technically i'am a van damme fan, or i was. this movie is so great that i hated myself for wasting those 90 minutes. do not let the name isaac florentine ( undisputed ii ) fool you, i had big hopes fo...
────────────────────────────────────────────────────────────────────────────────

────────────────────────────────────────────────────────────────────────────────
Method: SHAP | Edit Type: single-token
Changed Word: ('only', 'just')
Similarity: 0.9933 | Perplexity: 88.14

Original: enchanted april was on